In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [15]:
df = pd.read_json("dataV3.json")
df.shape

(7560, 29)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7560 entries, 0 to 7559
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Ano de Edição        3466 non-null   float64
 1   Colaborador          5077 non-null   object 
 2   Editora              5051 non-null   object 
 3   ISBN                 7228 non-null   object 
 4   Ano                  4997 non-null   float64
 5   Edição               4871 non-null   float64
 6   Origem               5075 non-null   object 
 7   Formato              5036 non-null   object 
 8   Encadernação         7196 non-null   object 
 9   Idioma               6291 non-null   object 
 10  País                 4974 non-null   object 
 11  Páginas              7053 non-null   float64
 12  Assunto              349 non-null    object 
 13  preço                7560 non-null   float64
 14  nome                 7560 non-null   object 
 15  categoria            7560 non-null   o

In [14]:
df.describe()

,Ano de Edição,Ano,Edição,Páginas,preço,Idade,EAN,Altura,Largura,Peso
count,3466.000000,4997.000000,4871.000000,7053.000000,7560.000000,184.000000,3.840000e+02,308.000000,308.000000,261.000000
mean,2015.345355,2014.425455,80.655923,284.348788,69.381878,7.456522,2.010324e+12,22.226429,16.385195,0.586596
std,6.349549,6.400459,391.598387,254.356544,74.818648,3.965460,6.047079e+09,11.323662,7.592012,0.936517
min,1966.000000,1966.000000,1.000000,1.000000,1.550000,1.000000,2.000018e+12,7.000000,5.000000,0.070000
25%,2013.000000,2012.000000,1.000000,144.000000,34.900000,4.000000,2.007100e+12,21.000000,14.000000,0.252500
50%,2017.000000,2016.000000,1.000000,224.000000,49.900000,6.500000,2.007200e+12,22.600000,16.000000,0.354000
75%,2020.000000,2019.000000,1.000000,336.000000,76.930000,11.000000,2.009500e+12,23.000000,16.000000,0.565000
max,2021.000000,2021.000000,2021.000000,4696.000000,1551.000000,19.000000,2.021123e+12,209.000000,133.000000,9.999000
